<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M6_Performing_a_Big_Data_workflow_with_Spark_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook will give a tutorial for starting out with PySpark using Titanic dataset. Let's get started. 


###  Goals

There are two primary goals of this kernel.
- Provide a tutorial for someone who is starting out with pyspark.
- Do an exploratory data analysis(EDA) of titanic with visualizations and storytelling.  


### What is Spark, anyway?
Spark is a distributed computing platform that enables the distribution of data and processing across clusters with multiple nodes. Each node in the cluster is akin to a separate computer, and splitting up data among nodes allows for efficient processing of very large datasets, with each node handling only a small portion of the data. The parallel nature of Spark's architecture allows each node to work on its own subset of the total data, enabling parallel processing of both data and computation across the nodes in the cluster. As a result, programming tasks that can be parallelized may be performed much faster.



Determining whether Spark is the most suitable solution for your problem requires some experience, but there are certain factors to consider, such as:

* Is the data too large to be processed on a single machine?
* Can the calculations be easily parallelized?



In [195]:
## installing pyspark
!pip install pyspark --q

In [196]:
## installing pyarrow
!pip install pyarrow --q

The first step in using Spark is connecting to a cluster. In practice, the cluster will be hosted on a remote machine that's connected to all other nodes. There will be one computer, called the master that manages splitting up the data and the computations. The master is connected to the rest of the computers in the cluster, which are called worker. The master sends the workers data and calculations to run, and they send their results back to the master.

We definitely don't need many clusters for the Titanic dataset. Moreover, the syntax for running Spark locally or on multiple clusters is quite similar. To begin working with Spark DataFrames, we need to create a SparkSession object from SparkContext. The SparkContext acts as the connection to the cluster, while the SparkSession serves as the interface for interacting with that connection. Let's create a SparkSession.

# Beginner Tutorial
This part is solely for beginners. I recommend starting from here to get a good understanding of the flow. 

In [197]:
## creating a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('tutorial').getOrCreate()

Let's read the dataset. 

In [198]:
import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv')

In [199]:
df.shape

(891, 12)

In [200]:
df_train = df[:600]
df_test = df[600:]

In [201]:
df_train.to_csv('titanic_train.csv', index=False)
df_test.to_csv('titanic_test.csv', index=False)

In [202]:
df_train = spark.read.csv('/content/titanic_train.csv', header = True, inferSchema=True)
df_test = spark.read.csv('/content/titanic_test.csv', header = True, inferSchema=True)

In [203]:
titanic_train = df_train.alias("titanic_train")

In [204]:
## So, what is df_train?
type(df_train)

pyspark.sql.dataframe.DataFrame

In [205]:
## As you can see it's a Spark dataframe. Let's take a look at the preview of the dataset. 
df_train.show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|null|    0|    0|      

In [206]:
## It looks a bit messi. See what I did there? ;). Anyway, how about using .toPandas() for change. 
df_train.toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S
...,...,...,...,...,...,...,...,...,...,...,...,...
595,596,0,3,"Van Impe, Mr. Jean Baptiste",male,36.0,1,1,345773,24.1500,None,S
596,597,1,2,"Leitch, Miss. Jessie Wills",female,NaN,0,0,248727,33.0000,None,S
597,598,0,3,"Johnson, Mr. Alfred",male,49.0,0,0,LINE,0.0000,None,S
598,599,0,3,"Boulos, Mr. Hanna",male,NaN,0,0,2664,7.2250,None,C


In [207]:
## how about a summary. 
result = df_train.describe().toPandas()

In [208]:
result

,summary,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,count,600,600,600,600,600,474,600,600,600,600,137,599
1,mean,300.5,0.3933333333333333,2.308333333333333,None,None,29.782700421940927,0.5383333333333333,0.375,272022.38051044085,31.846033999999975,None,None
2,stddev,173.34935823359717,0.4888973203772024,0.8353203911166349,None,None,14.535148472123723,1.0972102530757932,0.7737610593468783,501365.61217018607,46.2863007052945,None,None
3,min,1,0,1,"""Andersson, Mr. August Edvard (""""Wennerstrom"""")""",female,0.75,0,0,110152,0.0,A10,C
4,max,600,1,3,"van Billiard, Mr. Austin Blyler",male,71.0,8,5,WE/P 5735,512.3292,T,S


In [209]:
# getting the total row count
df_train.count()

600

In [210]:
# We can also convert a pandas dataframe to spark dataframe. Here is how we do it. 
print(f"Before: {type(result)}")
spark_temp = spark.createDataFrame(result)
print(f"After: {type(spark_temp)}")

Before: <class 'pandas.core.frame.DataFrame'>
After: <class 'pyspark.sql.dataframe.DataFrame'>


In [211]:
# Cool, Let's print the schema of the df using .printSchema()
df_train.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [212]:
# similar approach
df_train.dtypes

[('PassengerId', 'int'),
 ('Survived', 'int'),
 ('Pclass', 'int'),
 ('Name', 'string'),
 ('Sex', 'string'),
 ('Age', 'double'),
 ('SibSp', 'int'),
 ('Parch', 'int'),
 ('Ticket', 'string'),
 ('Fare', 'double'),
 ('Cabin', 'string'),
 ('Embarked', 'string')]

In the real world, data is rarely clean and often requires creating our own schema to process it. Speaking of schemas, you may be curious if Spark supports implementing SQL. The answer is yes, it does.

One of the main advantages of Spark is its ability to run SQL commands for data analysis. Let's take an example.

In [213]:
## First, we need to register a sql temporary view.
df_train.createOrReplaceTempView("mytable");

## Then, we use spark.sql and write sql inside it, which returns a spark Dataframe.  
result = spark.sql("SELECT * FROM mytable ORDER BY Fare DESC LIMIT 10")
result.toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,259,1,1,"Ward, Miss. Anna",female,35.0,0,0,PC 17755,512.3292,None,C
1,28,0,1,"Fortune, Mr. Charles Alexander",male,19.0,3,2,19950,263.0000,C23 C25 C27,S
2,89,1,1,"Fortune, Miss. Mabel Helen",female,23.0,3,2,19950,263.0000,C23 C25 C27,S
3,342,1,1,"Fortune, Miss. Alice Elizabeth",female,24.0,3,2,19950,263.0000,C23 C25 C27,S
4,439,0,1,"Fortune, Mr. Mark",male,64.0,1,4,19950,263.0000,C23 C25 C27,S
5,312,1,1,"Ryerson, Miss. Emily Borie",female,18.0,2,2,PC 17608,262.3750,B57 B59 B63 B66,C
6,300,1,1,"Baxter, Mrs. James (Helene DeLaudeniere Chaput)",female,50.0,0,1,PC 17558,247.5208,B58 B60,C
7,119,0,1,"Baxter, Mr. Quigg Edmond",male,24.0,0,1,PC 17558,247.5208,B58 B60,C
8,381,1,1,"Bidois, Miss. Rosalie",female,42.0,0,0,PC 17757,227.5250,None,C
9,558,0,1,"Robbins, Mr. Victor",male,NaN,0,0,PC 17757,227.5250,None,C


Similarly we can also register another sql temp view. 

In [214]:
df_test.createOrReplaceTempView("df_test")

Now that we have registered two tables with in this spark session, wondering how we can see which once are registered?

In [215]:
spark.catalog.listTables()

[Table(name='df_test', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='mytable', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [216]:
# similarly
spark.sql("SHOW views").show()

+---------+--------+-----------+
|namespace|viewName|isTemporary|
+---------+--------+-----------+
|         | df_test|       true|
|         | mytable|       true|
+---------+--------+-----------+



In [217]:
# We can also create spark dataframe out of these tables using spark.table
temp_table = spark.table("df_test")
print(type(temp_table))
temp_table.show(5)

<class 'pyspark.sql.dataframe.DataFrame'>
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|Ticket|  Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
|        601|       1|     2|Jacobsohn, Mrs. S...|female|24.0|    2|    1|243847|  27.0| null|       S|
|        602|       0|     3|Slabenoff, Mr. Petco|  male|null|    0|    0|349214|7.8958| null|       S|
|        603|       0|     1|Harrington, Mr. C...|  male|null|    0|    0|113796|  42.4| null|       S|
|        604|       0|     3|Torber, Mr. Ernst...|  male|44.0|    0|    0|364511|  8.05| null|       S|
|        605|       1|     1|"Homer, Mr. Harry...|  male|35.0|    0|    0|111426| 26.55| null|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+------+------+-----+--------+
only showing top 5 row

In [218]:
# What if want the column names only. 
df_train.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [219]:
# What about just a column?
df_train['Age']

Column<'Age'>

In [220]:
df_train.Age

Column<'Age'>

In [221]:
type(df_train['Age'])

pyspark.sql.column.Column

In [388]:
# Well, that's not what we pandas users have expected. 
# Yes, in order to get a column we need to use select().  
# df.select(df['Age']).show()
df_train.select('Age').show()

+----+
| Age|
+----+
|22.0|
|38.0|
|26.0|
|35.0|
|35.0|
|null|
|54.0|
| 2.0|
|27.0|
|14.0|
| 4.0|
|58.0|
|20.0|
|39.0|
|14.0|
|55.0|
| 2.0|
|null|
|31.0|
|null|
+----+
only showing top 20 rows



In [223]:
## What if we want multiple columns?
df_train.select(['Age', 'Fare']).show()

+----+-------+
| Age|   Fare|
+----+-------+
|22.0|   7.25|
|38.0|71.2833|
|26.0|  7.925|
|35.0|   53.1|
|35.0|   8.05|
|null| 8.4583|
|54.0|51.8625|
| 2.0| 21.075|
|27.0|11.1333|
|14.0|30.0708|
| 4.0|   16.7|
|58.0|  26.55|
|20.0|   8.05|
|39.0| 31.275|
|14.0| 7.8542|
|55.0|   16.0|
| 2.0| 29.125|
|null|   13.0|
|31.0|   18.0|
|null|  7.225|
+----+-------+
only showing top 20 rows



In [224]:
# similarly 
df_train[['Age', 'Fare']].show()

+----+-------+
| Age|   Fare|
+----+-------+
|22.0|   7.25|
|38.0|71.2833|
|26.0|  7.925|
|35.0|   53.1|
|35.0|   8.05|
|null| 8.4583|
|54.0|51.8625|
| 2.0| 21.075|
|27.0|11.1333|
|14.0|30.0708|
| 4.0|   16.7|
|58.0|  26.55|
|20.0|   8.05|
|39.0| 31.275|
|14.0| 7.8542|
|55.0|   16.0|
| 2.0| 29.125|
|null|   13.0|
|31.0|   18.0|
|null|  7.225|
+----+-------+
only showing top 20 rows



In [225]:
# or 
df_train[df_train.Age, 
         df_train.Fare].show()

+----+-------+
| Age|   Fare|
+----+-------+
|22.0|   7.25|
|38.0|71.2833|
|26.0|  7.925|
|35.0|   53.1|
|35.0|   8.05|
|null| 8.4583|
|54.0|51.8625|
| 2.0| 21.075|
|27.0|11.1333|
|14.0|30.0708|
| 4.0|   16.7|
|58.0|  26.55|
|20.0|   8.05|
|39.0| 31.275|
|14.0| 7.8542|
|55.0|   16.0|
| 2.0| 29.125|
|null|   13.0|
|31.0|   18.0|
|null|  7.225|
+----+-------+
only showing top 20 rows



The syntax for Pyspark DataFrame is simple, as you can see, with multiple ways of implementation. The choice of the best syntax depends on what we aim to achieve, and I will elaborate more on this later. Let us now see how we can access a row.

In [390]:
df_train.head(2)

[Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Fare=7.25, Cabin='G', Embarked='S', name_length=23, nLength_group='medium', title='Mr', family_size=1, family_group='loner', is_alone=1, calculated_fare=7.25, fare_group='low'),
 Row(PassengerId=2, Survived=1, Pclass=1, Name='Cumings, Mrs. John Bradley (Florence Briggs Thayer)', Sex='female', Age=38.0, SibSp=1, Parch=0, Fare=71.2833, Cabin='C', Embarked='C', name_length=51, nLength_group='long', title='Mrs', family_size=1, family_group='loner', is_alone=1, calculated_fare=71.2833, fare_group='very_high')]

In [391]:
type(df_train.head(1))

list

In [392]:
## returns a list. let's get the item in the list
row = df_train.head(1)[0]
row

Row(PassengerId=1, Survived=0, Pclass=3, Name='Braund, Mr. Owen Harris', Sex='male', Age=22.0, SibSp=1, Parch=0, Fare=7.25, Cabin='G', Embarked='S', name_length=23, nLength_group='medium', title='Mr', family_size=1, family_group='loner', is_alone=1, calculated_fare=7.25, fare_group='low')

In [393]:
type(row)

pyspark.sql.types.Row

In [394]:
## row can be converted into dict using .asDict()
row.asDict()

{'PassengerId': 1,
 'Survived': 0,
 'Pclass': 3,
 'Name': 'Braund, Mr. Owen Harris',
 'Sex': 'male',
 'Age': 22.0,
 'SibSp': 1,
 'Parch': 0,
 'Fare': 7.25,
 'Cabin': 'G',
 'Embarked': 'S',
 'name_length': 23,
 'nLength_group': 'medium',
 'title': 'Mr',
 'family_size': 1,
 'family_group': 'loner',
 'is_alone': 1,
 'calculated_fare': 7.25,
 'fare_group': 'low'}

In [395]:
## Then the value can be accessed from the row dictionaly. 
row.asDict()['PassengerId']

1

In [396]:
## similarly
row.asDict()['Name']

'Braund, Mr. Owen Harris'

In [233]:
## let's say we want to change the name of a column. we can use withColumnRenamed
# df.withColumnRenamed('exsisting name', 'anticipated name');
df_train.withColumnRenamed("Age", "newA").limit(5).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,newA,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


In [234]:
# Let's say we want to modify a column, for example, add in this case, adding $20 with every fare. 
## df.withColumn('existing column', 'calculation with the column(we have to put df not just column)')
## so not df.withColumn('Fare', 'Fare' +20).show()
df_train.withColumn('Fare', df_train['Fare']+20).limit(5).show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|  27.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|91.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282| 27.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   73.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|  28.05| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

Now this change isn't permanent since we are not assigning it to any variables. 

In [235]:
## let's say we want to get the average fare.
# we will use the "mean" function from pyspark.sql.functions(this is where all the functions are stored) and
# collect the data using ".collect()" instead of using .show()
# collect returns a list so we need to get the value from the list using index

In [236]:
from pyspark.sql.functions import mean
fare_mean = df_train.select(mean("Fare")).collect()
fare_mean[0][0]

31.846033999999975

In [237]:
fare_mean = fare_mean[0][0]
fare_mean

31.846033999999975

#### Filter

In [238]:
# What if we want to filter data and see all fare above average. 
# there are two approaches of this, we can use sql syntex/passing a string
# or just dataframe approach. 
df_train.filter("Fare > 32.20" ).limit(3).show()

+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|PC 17599|71.2833|  C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|  113803|   53.1| C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|   17463|51.8625|  E46|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+



In [239]:
# similarly 
df_train[df_train.Fare > 32.20].limit(3).show()

+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|PC 17599|71.2833|  C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|  113803|   53.1| C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|   17463|51.8625|  E46|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+



In [240]:
# or we can use the dataframe approach
df_train.filter(df_train['Fare'] > fare_mean).limit(3).show()

+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|PC 17599|71.2833|  C85|       C|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|  113803|   53.1| C123|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|   17463|51.8625|  E46|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-------+-----+--------+



In [241]:
## What if we want to filter by multiple columns.
# passenger with below average fare with a sex equals male
temp_df = df_train.filter((df_train['Fare'] < fare_mean) &
          (df_train['Sex'] ==  'male')
         )
temp_df.show(5)

+-----------+--------+------+--------------------+----+----+-----+-----+---------+------+-----+--------+
|PassengerId|Survived|Pclass|                Name| Sex| Age|SibSp|Parch|   Ticket|  Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+---------+------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|male|22.0|    1|    0|A/5 21171|  7.25| null|       S|
|          5|       0|     3|Allen, Mr. Willia...|male|35.0|    0|    0|   373450|  8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|male|null|    0|    0|   330877|8.4583| null|       Q|
|          8|       0|     3|Palsson, Master. ...|male| 2.0|    3|    1|   349909|21.075| null|       S|
|         13|       0|     3|Saundercock, Mr. ...|male|20.0|    0|    0|A/5. 2151|  8.05| null|       S|
+-----------+--------+------+--------------------+----+----+-----+-----+---------+------+-----+--------+
only showing top 5 rows



In [242]:
# similarly 
df_train[(df_train.Fare < fare_mean) & 
         (df_train.Sex == "male")].show(5)

+-----------+--------+------+--------------------+----+----+-----+-----+---------+------+-----+--------+
|PassengerId|Survived|Pclass|                Name| Sex| Age|SibSp|Parch|   Ticket|  Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+---------+------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|male|22.0|    1|    0|A/5 21171|  7.25| null|       S|
|          5|       0|     3|Allen, Mr. Willia...|male|35.0|    0|    0|   373450|  8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|male|null|    0|    0|   330877|8.4583| null|       Q|
|          8|       0|     3|Palsson, Master. ...|male| 2.0|    3|    1|   349909|21.075| null|       S|
|         13|       0|     3|Saundercock, Mr. ...|male|20.0|    0|    0|A/5. 2151|  8.05| null|       S|
+-----------+--------+------+--------------------+----+----+-----+-----+---------+------+-----+--------+
only showing top 5 rows



In [243]:
# passenger with below average fare and are not male
filter1_less_than_mean_fare = df_train['Fare'] < fare_mean
filter2_sex_not_male = df_train['Sex'] != "male"
df_train.filter((filter1_less_than_mean_fare) &
                (filter2_sex_not_male)).show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female|27.0|    0|    2|          347742|11.1333| null|       S|
|         10|       1|     2|Nasser, Mrs. Nich...|female|14.0|    1|    0|          237736|30.0708| null|       C|
|         11|       1|     3|Sandstrom, Miss. ...|female| 4.0|    1|    1|         PP 9549|   16.7|   G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|58.0|    0|    0|          113783|  26.55| C103|       S|
|         15|       0|     3|Vestrom, Miss. Hu...|female|14.0|    0|    0|      

In [244]:
# We can also apply it this way
# passenger with below fare and are not male
# creating filters
filter1_less_than_mean_fare = df_train['Fare'] < fare_mean
filter2_sex_not_male = df_train['Sex'] != "male"
# applying filters
df_train.filter(filter1_less_than_mean_fare).filter(filter2_sex_not_male).show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female|27.0|    0|    2|          347742|11.1333| null|       S|
|         10|       1|     2|Nasser, Mrs. Nich...|female|14.0|    1|    0|          237736|30.0708| null|       C|
|         11|       1|     3|Sandstrom, Miss. ...|female| 4.0|    1|    1|         PP 9549|   16.7|   G6|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|58.0|    0|    0|          113783|  26.55| C103|       S|
|         15|       0|     3|Vestrom, Miss. Hu...|female|14.0|    0|    0|      

In [245]:
# we can also filter by using builtin functions.
# between
df_train.select("PassengerId", "Fare").filter(df_train.Fare.between(10,40)).show()

+-----------+-------+
|PassengerId|   Fare|
+-----------+-------+
|          8| 21.075|
|          9|11.1333|
|         10|30.0708|
|         11|   16.7|
|         12|  26.55|
|         14| 31.275|
|         16|   16.0|
|         17| 29.125|
|         18|   13.0|
|         19|   18.0|
|         21|   26.0|
|         22|   13.0|
|         24|   35.5|
|         25| 21.075|
|         26|31.3875|
|         31|27.7208|
|         34|   10.5|
|         39|   18.0|
|         40|11.2417|
|         42|   21.0|
+-----------+-------+
only showing top 20 rows



In [246]:
df_train.select("PassengerID", df_train.Fare.between(10,40)).show()

+-----------+-------------------------------+
|PassengerID|((Fare >= 10) AND (Fare <= 40))|
+-----------+-------------------------------+
|          1|                          false|
|          2|                          false|
|          3|                          false|
|          4|                          false|
|          5|                          false|
|          6|                          false|
|          7|                          false|
|          8|                           true|
|          9|                           true|
|         10|                           true|
|         11|                           true|
|         12|                           true|
|         13|                          false|
|         14|                           true|
|         15|                          false|
|         16|                           true|
|         17|                           true|
|         18|                           true|
|         19|                     

In [247]:
# contains
df_train.select("PassengerId", "Name").filter(df_train.Name.contains("Mr")).show()

+-----------+--------------------+
|PassengerId|                Name|
+-----------+--------------------+
|          1|Braund, Mr. Owen ...|
|          2|Cumings, Mrs. Joh...|
|          4|Futrelle, Mrs. Ja...|
|          5|Allen, Mr. Willia...|
|          6|    Moran, Mr. James|
|          7|McCarthy, Mr. Tim...|
|          9|Johnson, Mrs. Osc...|
|         10|Nasser, Mrs. Nich...|
|         13|Saundercock, Mr. ...|
|         14|Andersson, Mr. An...|
|         16|Hewlett, Mrs. (Ma...|
|         18|Williams, Mr. Cha...|
|         19|Vander Planke, Mr...|
|         20|Masselmani, Mrs. ...|
|         21|Fynney, Mr. Joseph J|
|         22|Beesley, Mr. Lawr...|
|         24|Sloper, Mr. Willi...|
|         26|Asplund, Mrs. Car...|
|         27|Emir, Mr. Farred ...|
|         28|Fortune, Mr. Char...|
+-----------+--------------------+
only showing top 20 rows



In [248]:
# startswith 
df_train.select("PassengerID", 'Sex').filter(df_train.Sex.startswith("fe")).show()

+-----------+------+
|PassengerID|   Sex|
+-----------+------+
|          2|female|
|          3|female|
|          4|female|
|          9|female|
|         10|female|
|         11|female|
|         12|female|
|         15|female|
|         16|female|
|         19|female|
|         20|female|
|         23|female|
|         25|female|
|         26|female|
|         29|female|
|         32|female|
|         33|female|
|         39|female|
|         40|female|
|         41|female|
+-----------+------+
only showing top 20 rows



In [249]:
# endswith
df_train.select("PassengerID", 'Ticket').filter(df_train.Ticket.endswith("50")).show()

+-----------+------+
|PassengerID|Ticket|
+-----------+------+
|          5|373450|
|         28| 19950|
|         89| 19950|
|        256|  2650|
|        342| 19950|
|        439| 19950|
|        537|113050|
+-----------+------+



In [250]:
# isin
df_train[df_train.PassengerId.isin([1,2,3])].show()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+



In [251]:
# like
df_train[df_train.Name.like("Br%")].show()

+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|A/5 21171|   7.25| null|       S|
|        195|       1|     1|Brown, Mrs. James...|female|44.0|    0|    0| PC 17610|27.7208|   B4|       C|
|        222|       0|     2|Bracken, Mr. James H|  male|27.0|    0|    0|   220367|   13.0| null|       S|
|        478|       0|     3|Braund, Mr. Lewis...|  male|29.0|    1|    0|     3460| 7.0458| null|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+



In [252]:
# substr
df_train.select(df_train.Name.substr(1,5)).show()

+---------------------+
|substring(Name, 1, 5)|
+---------------------+
|                Braun|
|                Cumin|
|                Heikk|
|                Futre|
|                Allen|
|                Moran|
|                McCar|
|                Palss|
|                Johns|
|                Nasse|
|                Sands|
|                Bonne|
|                Saund|
|                Ander|
|                Vestr|
|                Hewle|
|                Rice,|
|                Willi|
|                Vande|
|                Masse|
+---------------------+
only showing top 20 rows



In [253]:
# similarly 
df_train[[df_train.Name.substr(1,5)]].show()

+---------------------+
|substring(Name, 1, 5)|
+---------------------+
|                Braun|
|                Cumin|
|                Heikk|
|                Futre|
|                Allen|
|                Moran|
|                McCar|
|                Palss|
|                Johns|
|                Nasse|
|                Sands|
|                Bonne|
|                Saund|
|                Ander|
|                Vestr|
|                Hewle|
|                Rice,|
|                Willi|
|                Vande|
|                Masse|
+---------------------+
only showing top 20 rows



One interesting thing about substr method is that we can't implement the following syntax while working with substr. This syntax is best implemented in a filter when the return values are boolean not a column.

In [254]:
# df_train[df_train.Name.substr(1,5)].show()

#### GroupBy

In [255]:
## Let's group by Pclass and get the average fare price per Pclass.  
df_train.groupBy("Pclass").mean().toPandas()

,Pclass,avg(PassengerId),avg(Survived),avg(Pclass),avg(Age),avg(SibSp),avg(Parch),avg(Fare)
0,1,323.813793,0.593103,1.0,38.379360,0.441379,0.400000,84.436120
1,3,288.269697,0.263636,3.0,25.189914,0.648485,0.375758,13.315656
2,2,305.744000,0.504000,2.0,29.744224,0.360000,0.344000,19.761733


In [256]:
## let's just look at the Pclass and avg(Fare)
df_train.groupBy("Pclass").mean().select('Pclass', 'avg(Fare)').show()

+------+------------------+
|Pclass|         avg(Fare)|
+------+------------------+
|     1| 84.43611999999995|
|     3| 13.31565575757576|
|     2|19.761732799999997|
+------+------------------+



In [257]:
# Alternative way
df_train.groupBy("Pclass").mean("Fare").show()

+------+------------------+
|Pclass|         avg(Fare)|
+------+------------------+
|     1| 84.43611999999995|
|     3| 13.31565575757576|
|     2|19.761732799999997|
+------+------------------+



In [258]:
## What if we want just the average of all fare, we can use .agg with the dataframe. 
df_train.agg({'Fare':'mean'}).show()

+------------------+
|         avg(Fare)|
+------------------+
|31.846033999999975|
+------------------+



In [259]:
## another way this can be done is by importing "mean" funciton from pyspark.sql.functions
from pyspark.sql.functions import mean
df_train.select(mean("Fare")).show()

+------------------+
|         avg(Fare)|
+------------------+
|31.846033999999975|
+------------------+



In [260]:
## we can also combine the few previous approaches to get similar results. 
temp = df_train.groupBy("Pclass")
temp.agg({"Fare": 'mean'}).show()

+------+------------------+
|Pclass|         avg(Fare)|
+------+------------------+
|     1| 84.43611999999995|
|     3| 13.31565575757576|
|     2|19.761732799999997|
+------+------------------+



In [261]:
# What if we want to format the results. 
# for example,
# I want to rename the column. this will be accomplished using .alias() method.  
# I want to format the number with only two decimals. this can be done using "format_number"
from pyspark.sql.functions import format_number
temp = df_train.groupBy("Pclass")
temp = temp.agg({"Fare": 'mean'})
temp.select('Pclass', format_number("avg(Fare)", 2).alias("average fare")).show()

+------+------------+
|Pclass|average fare|
+------+------------+
|     1|       84.44|
|     3|       13.32|
|     2|       19.76|
+------+------------+



#### OrderBy
There are many built in functions that we can use to do orderby in spark. Let's look at some of those. 

In [262]:
## What if I want to order by Fare in ascending order. 
df_train.orderBy("Fare").limit(20).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,278,0,2,"""Parkes, Mr. Francis """"Frank""""""",male,NaN,0,0,239853,0.0000,None,S
1,303,0,3,"Johnson, Mr. William Cahoone Jr",male,19.0,0,0,LINE,0.0000,None,S
2,482,0,2,"""Frost, Mr. Anthony Wood """"Archie""""""",male,NaN,0,0,239854,0.0000,None,S
3,180,0,3,"Leonard, Mr. Lionel",male,36.0,0,0,LINE,0.0000,None,S
4,414,0,2,"Cunningham, Mr. Alfred Fleming",male,NaN,0,0,239853,0.0000,None,S
5,467,0,2,"Campbell, Mr. William",male,NaN,0,0,239853,0.0000,None,S
6,272,1,3,"Tornquist, Mr. William Henry",male,25.0,0,0,LINE,0.0000,None,S
7,598,0,3,"Johnson, Mr. Alfred",male,49.0,0,0,LINE,0.0000,None,S
8,264,0,1,"Harrison, Mr. William",male,40.0,0,0,112059,0.0000,B94,S
9,379,0,3,"Betros, Mr. Tannous",male,20.0,0,0,2648,4.0125,None,C


In [263]:
# similarly
df_train.orderBy(df_train.Fare.asc()).show()

+-----------+--------+------+--------------------+----+----+-----+-----+------------------+------+-----+--------+
|PassengerId|Survived|Pclass|                Name| Sex| Age|SibSp|Parch|            Ticket|  Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+------------------+------+-----+--------+
|        278|       0|     2|"Parkes, Mr. Fran...|male|null|    0|    0|            239853|   0.0| null|       S|
|        180|       0|     3| Leonard, Mr. Lionel|male|36.0|    0|    0|              LINE|   0.0| null|       S|
|        272|       1|     3|Tornquist, Mr. Wi...|male|25.0|    0|    0|              LINE|   0.0| null|       S|
|        414|       0|     2|Cunningham, Mr. A...|male|null|    0|    0|            239853|   0.0| null|       S|
|        467|       0|     2|Campbell, Mr. Wil...|male|null|    0|    0|            239853|   0.0| null|       S|
|        264|       0|     1|Harrison, Mr. Wil...|male|40.0|    0|    0|            1120

In [264]:
# What about descending order
# df.orderBy(df['Fare'].desc()).limit(5).show()
# dot notation
df_train.orderBy(df_train.Fare.desc()).limit(5).show()

+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|  Ticket|    Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+--------+-----------+--------+
|        259|       1|     1|    Ward, Miss. Anna|female|35.0|    0|    0|PC 17755|512.3292|       null|       C|
|         89|       1|     1|Fortune, Miss. Ma...|female|23.0|    3|    2|   19950|   263.0|C23 C25 C27|       S|
|         28|       0|     1|Fortune, Mr. Char...|  male|19.0|    3|    2|   19950|   263.0|C23 C25 C27|       S|
|        342|       1|     1|Fortune, Miss. Al...|female|24.0|    3|    2|   19950|   263.0|C23 C25 C27|       S|
|        439|       0|     1|   Fortune, Mr. Mark|  male|64.0|    1|    4|   19950|   263.0|C23 C25 C27|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+--------+-----

In [265]:
df_train.filter(df_train.Embarked.isNull()).count()

1

In [266]:
df_train.select('PassengerID','Embarked').orderBy(df_train.Embarked.asc_nulls_first()).show()

+-----------+--------+
|PassengerID|Embarked|
+-----------+--------+
|         62|    null|
|        204|       C|
|         66|       C|
|         74|       C|
|         31|       C|
|         97|       C|
|         65|       C|
|         98|       C|
|         53|       C|
|        112|       C|
|         35|       C|
|        115|       C|
|         40|       C|
|        119|       C|
|         44|       C|
|        123|       C|
|        126|       C|
|         58|       C|
|        129|       C|
|          2|       C|
+-----------+--------+
only showing top 20 rows



In [267]:
df_train.select('PassengerID','Embarked').orderBy(df_train.Embarked.asc_nulls_last()).tail(5)

[Row(PassengerID=595, Embarked='S'),
 Row(PassengerID=596, Embarked='S'),
 Row(PassengerID=597, Embarked='S'),
 Row(PassengerID=598, Embarked='S'),
 Row(PassengerID=62, Embarked=None)]

In [268]:
## How do we deal with missing values. 
# df.na.drop(how=("any"/"all"), thresh=(1,2,3,4,5...))
df_train.na.drop(how="any").limit(5).toPandas()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
1,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
2,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
3,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
4,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S


<img src="https://raw.githubusercontent.com/aaubs/ds-master/main/data/Images/Exercise.png" width="600">